### Overview

- Demonstration of various VTK datasets
- Uses TVTK: which is much easier than raw VTK
- Also uses mlab for visualization


In [ ]:
%gui qt

In [ ]:
from IPython.display import Image
from tvtk.api import tvtk
from mayavi import mlab
import numpy as np
from numpy import array

In [ ]:
fig = mlab.figure(bgcolor=(1,1,1), fgcolor=(0,0,0))

### Structured Points: 2D


In [ ]:
from scipy import special
# The scalar values.
x = (np.arange(50.0) - 25)/2.0
y = x.copy()
r = np.sqrt(x[:,None]**2+y**2)
#x, y = np.mgrid[-12.5:12.5:25j, -12.5:12.5:25j]
#r = np.sqrt(x*x + y*y)
z = 5.0*special.j0(r)  # Bessel function of order 0

In [ ]:
# --------------------------------------------------
# Can't specify explicit points, they are implicit.
# The volume specified using origin, spacing and dims.
from tvtk.api import tvtk
spoints = tvtk.StructuredPoints(origin=(-12.5,-12.5,0),
                                spacing=(0.5,0.5,1),
                                dimensions=(50,50,1))

# Transpose the array data due to VTK's implicit
# ordering. We flatten it so the number of components
# is 1.
spoints.point_data.scalars = z.T.flatten()
spoints.point_data.scalars.name = 'scalar'

### Visualizing it


In [ ]:
# Add the dataset to the pipeline
src = mlab.pipeline.add_dataset(spoints)

warp = mlab.pipeline.warp_scalar(src)
surf = mlab.pipeline.surface(warp)
mlab.savefig('structured_points2d.png')
Image('structured_points2d.png')

### Structured Points: 3D


In [ ]:
x, y, z = np.ogrid[-5:5:128j,-5:5:128j,
                   -5:5:128j]
x, y, z = [t.astype('f') for t in (x, y, z)]
scalars = np.sin(x*y*z)/(x*y*z)

# --------------------------------------------------
spoints = tvtk.StructuredPoints(
    origin=(-5.,-5,-5),
    spacing=(10./127,10./127,10./127),
    dimensions=(128,128,128)
)
# The copy makes the data contiguous and the transpose
# makes it suitable for display via tvtk.
s = scalars.transpose().copy()
spoints.point_data.scalars = np.ravel(s)
spoints.point_data.scalars.name = 'scalars'

### Visualizing it


In [ ]:
mlab.clf()
# Add the dataset to the pipeline
src = mlab.pipeline.add_dataset(spoints)

cut = mlab.pipeline.scalar_cut_plane(src)
contour = mlab.pipeline.iso_surface(src)

mlab.savefig('structured_points3d.png')
Image('structured_points3d.png')

### Structured Grid


In [ ]:
r, th, z = np.mgrid[1:10:25j, 0:2*np.pi:51j, 0:5:25j]
x, y = np.cos(th)*r, np.sin(th)*r
scalar = x*x + y*y + z*z

In [ ]:
pts = np.empty(z.shape + (3,))
pts[...,0] = x
pts[...,1] = y
pts[...,2] = z

In [ ]:
pts = pts.transpose(2, 1, 0, 3).copy()
pts.shape = pts.size//3, 3

In [ ]:
sgrid = tvtk.StructuredGrid(dimensions=x.shape)
sgrid.points = pts
sgrid.point_data.scalars = np.ravel(scalar.T.copy())
sgrid.point_data.scalars.name = 'scalars'

### Visualizing it


In [ ]:
mlab.clf()
# Add the dataset to the pipeline
src = mlab.pipeline.add_dataset(sgrid)

plane = mlab.pipeline.grid_plane(src)
plane.grid_plane.axis = 'z'
c_plane = mlab.pipeline.contour_grid_plane(src)
c_plane.enable_contours = False
iso = mlab.pipeline.iso_surface(src)

mlab.savefig('structured_grid.png')
Image('structured_grid.png')

### PolyData


In [ ]:
# The points in 3D.
points = array([[0.,0,0], [1,0,0], [0,1,0], [0,0,1]])
# Connectivity via indices to the points.
triangles = array([[0,1,3], [0,3,2], [1,2,3], [0,2,1]])
# Creating the data object.

mesh = tvtk.PolyData()
mesh.points = points # the points
mesh.polys = triangles # triangles for connectivity.
# For lines/verts: mesh.lines = lines; mesh.verts = verts
# Now create some point data.
temperature = array([10., 20. ,30., 40.], 'f')
mesh.point_data.scalars = temperature
mesh.point_data.scalars.name = 'temperature'
# Some vectors.
velocity = array([[0.,0.,0], [1.,0,0], [0.,1,0], [0.,0,1]])
mesh.point_data.vectors = velocity
mesh.point_data.vectors.name = 'velocity'

### Visualizing it


In [ ]:
mlab.clf()

# Add the dataset to the pipeline
src = mlab.pipeline.add_dataset(mesh)

surf = mlab.pipeline.surface(src)
vec = mlab.pipeline.vectors(src)

mlab.savefig('polydata.png')
Image('polydata.png')

### Unstructured Grid


In [ ]:
points = array([[0.,0.,0], [1.,0,0], [0.,1,0], [0.,0,1]])
tets = array([[0, 1, 2, 3]])
tet_type = tvtk.Tetra().cell_type # VTK_TETRA == 10
#--------------------------------------------------
ug = tvtk.UnstructuredGrid(points=points)
# This sets up the cells.
ug.set_cells(tet_type, tets)
# Attribute data.
temperature = array([10, 20 ,20, 30], 'f')
ug.point_data.scalars = temperature
ug.point_data.scalars.name = 'temperature'
# Some vectors.
velocity = array([[0.,0,0], [1,0,0], [0,1,0], [0,0,1]])
ug.point_data.vectors = velocity
ug.point_data.vectors.name = 'velocity'

### Visualizing it


In [ ]:
mlab.clf()

# Add the dataset to the pipeline
src = mlab.pipeline.add_dataset(ug)

surf = mlab.pipeline.surface(src)
vec = mlab.pipeline.vectors(src)

mlab.savefig('ug.png')
Image('ug.png')

### Writing the data out


In [ ]:
from tvtk.api import write_data
write_data(ug, ’ug’)
write_data(mesh, ’mesh’)